EXAM: HEALTHCARE INFORMATION SYSTEM \
NAME: Eleonora Zucchi \
\
I. ETL Exercise

In [2]:
import pandas as pd
import os.path
import psycopg2
import numpy as np
from  datetime import datetime

True. Check job parmeters 

Parameters:
  - partecipant
  - day
  - group minutes

In [3]:
file_path = '/Users/eleonorazucchi/Desktop/HIS/pmdata/participant-overview.xlsx'

participants = pd.read_excel(file_path, header=1) 
print(participants)

   Participant ID  Age  Height  Gender A or B person Max heart rate  \
0             p01   48     195    male             A            182   
1             p02   60     180    male             A            169   
2             p03   25     184    male             A            157   
3             p04   26     163  female             A            195   
4             p05   35     176    male             A            184   
5             p06   42     179   male              B            181   
6             p07   26     177    male             B            NaN   
7             p08   27     186    male             B            200   
8             p09   26     180    male             B            183   
9             p10   38     179  female             B            197   
10            p11   25     171  female             A            203   
11            p12   27     178    male             A                  
12            p13   31     183    male             A            186   
13    

In [4]:
participants = participants['Participant ID'][0]
print(participants)

p01


In [5]:
group_minutes = 5

In [6]:
day = '2019-11-01'

2. Tables preparation

Function that creates the database table group_{group_minutes}_values

In [7]:
def create_table(group_minutes: int):
    command = """
            CREATE TABLE IF NOT EXISTS public.group_{}_values (
                partecipant character varying(5),
                VT timestamp without time zone,
                calories bigint,
                bpm bigint,
                walking boolean,
                swimming boolean,
                running boolean,
                cycling boolean,
                treadmill boolean,
                PRIMARY KEY (partecipant, vt)
            )
            """.format(str(group_minutes))

    connection = None
    try:
        # connect to the PostgreSQL server
        connection = psycopg2.connect(
            database='postgres', 
            user='postgres', 
            password='pwd'
        )
        cur = connection.cursor()

        cur.execute(command)
        cur.close()
        
        # commit the changes
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    finally:
        if connection is not None:
            connection.close()
            
    return

create_table(group_minutes=group_minutes)

3. Rows allocation

In [8]:

def load_data(participants: list, table_name: str, index: str):
    dfs = []

    #for participant in participants:

    path = os.path.join('pmdata', participants, 'fitbit', table_name + '.json')

    df = pd.read_json(path)
    df = df.set_index(index)
    df['participant'] = participants
    dfs.append(df) 

    dataframe = pd.concat(dfs)
    return dataframe


In [9]:
calories = load_data(
    participants=participants, 
    table_name='calories', 
    index='dateTime'
)

print(calories)

                     value participant
dateTime                              
2019-11-01 00:00:00   1.39         p01
2019-11-01 00:01:00   1.39         p01
2019-11-01 00:02:00   1.39         p01
2019-11-01 00:03:00   1.39         p01
2019-11-01 00:04:00   1.39         p01
...                    ...         ...
2020-03-31 23:55:00   1.38         p01
2020-03-31 23:56:00   1.38         p01
2020-03-31 23:57:00   1.38         p01
2020-03-31 23:58:00   1.52         p01
2020-03-31 23:59:00   3.31         p01

[218880 rows x 2 columns]


In [10]:
bpm = load_data(participants=participants, table_name='heart_rate', index='dateTime')
bpm["value"] = bpm["value"].apply(lambda x: x['bpm']) 

print(bpm)

                     value participant
dateTime                              
2019-11-01 00:00:05     54         p01
2019-11-01 00:00:10     52         p01
2019-11-01 00:00:20     51         p01
2019-11-01 00:00:25     52         p01
2019-11-01 00:00:30     51         p01
...                    ...         ...
2020-03-31 23:59:18     55         p01
2020-03-31 23:59:23     54         p01
2020-03-31 23:59:33     56         p01
2020-03-31 23:59:48     56         p01
2020-03-31 23:59:58     55         p01

[1573165 rows x 2 columns]


In [11]:
exercises = load_data(
    participants=participants, 
    table_name='exercise', 
    index='startTime'
)

exercises.index = pd.to_datetime(exercises.index, infer_datetime_format=True)
exercises = exercises[['participant', 'activityName', 'activeDuration']]

print(exercises)

                    participant activityName  activeDuration
startTime                                                   
2019-11-01 14:56:32         p01         Walk         1331000
2019-11-01 19:03:11         p01         Walk         2202000
2019-11-02 13:26:38         p01         Walk         2458000
2019-11-04 21:22:08         p01         Walk         1024000
2019-11-05 19:27:25         p01         Walk          973000
...                         ...          ...             ...
2020-03-27 13:07:53         p01         Walk         1076000
2020-03-27 16:22:27         p01         Walk         2918000
2020-03-28 09:58:08         p01         Walk         5581000
2020-03-29 07:42:53         p01         Walk         1076000
2020-03-31 16:51:36         p01         Walk         2764000

[190 rows x 3 columns]


4. - Calories aggregate & insert
   -  BPM aggregate & insert
   -  Exercises transform & insert

Function that normalizes the data setting a common granularity and ending date. 
Values are summed according to the choosen timestamp, 
missing values are copied from the previous row

In [12]:
def aggregate_sum(dataframe, group_minutes: int, day: str):

    freq = str(group_minutes) + 'T'

    date = pd.to_datetime(day)
    new_date = pd.to_datetime(date + pd.Timedelta('1 day'))
    
    dataframe = dataframe.iloc[dataframe.index >= date]
    dataframe = dataframe.iloc[dataframe.index < new_date]

    dataframe.index = dataframe.index.round(freq=freq)
    dataframe = dataframe.groupby(by=dataframe.index, as_index=1).agg({'value': 'sum', 'participant': 'last'})
    dataframe = dataframe.resample(freq).ffill()
  
    return dataframe

Function that normalizes the data setting a common granularity and ending date. 
Values are averaged according to the choosen timestamp, 
missing values are copied from the previous row

In [13]:
def aggregate_mean(dataframe, group_minutes: int, day: str):

    freq = str(group_minutes) + 'T'

    date = pd.to_datetime(day)
    new_date = pd.to_datetime(date + pd.Timedelta('1 day'))

    dataframe = dataframe.iloc[dataframe.index >= date]

    dataframe = dataframe.iloc[dataframe.index < new_date]

    dataframe.index = dataframe.index.round(freq=freq)
    dataframe = dataframe.groupby(by=dataframe.index, as_index=1).agg({'value': 'mean', 'participant': 'last'})
    dataframe = dataframe.resample(freq).ffill()
       
    return dataframe  

In [14]:
def aggregate_copy(dataframe, group_minutes: int, day: str):

    freq = str(group_minutes) + 'T'
            
    date = pd.to_datetime(day)
    new_date = pd.to_datetime(date + pd.Timedelta('1 day'))

    dataframe = dataframe.iloc[dataframe.index >= date]
    dataframe = dataframe.iloc[dataframe.index < new_date]

    dataframe.index = dataframe.index.round(freq=freq)
    dataframe = dataframe.groupby(by=dataframe.index, as_index=1).agg({'activeDuration': 'last','activityName': 'last', 'participant': 'last'})
    #dataframe.index = dataframe.index.resample(freq).ffill() 
        
    return dataframe

In [15]:
calories_aggregated = aggregate_sum(
    dataframe=calories, 
    group_minutes=group_minutes, 
    day=day
)

print(calories_aggregated)

                     value participant
dateTime                              
2019-11-01 00:00:00   4.17         p01
2019-11-01 00:05:00   6.95         p01
2019-11-01 00:10:00   6.95         p01
2019-11-01 00:15:00   6.95         p01
2019-11-01 00:20:00   6.95         p01
...                    ...         ...
2019-11-01 23:40:00   6.95         p01
2019-11-01 23:45:00  11.26         p01
2019-11-01 23:50:00  24.87         p01
2019-11-01 23:55:00  22.52         p01
2019-11-02 00:00:00   2.92         p01

[289 rows x 2 columns]


In [16]:
bpm_aggregated = aggregate_mean(
    dataframe=bpm, 
    group_minutes=group_minutes, 
    day=day
)

print(bpm_aggregated)

                         value participant
dateTime                                  
2019-11-01 00:00:00  53.000000         p01
2019-11-01 00:05:00  53.488889         p01
2019-11-01 00:10:00  53.277778         p01
2019-11-01 00:15:00  54.171429         p01
2019-11-01 00:20:00  54.000000         p01
...                        ...         ...
2019-11-01 23:40:00  55.850000         p01
2019-11-01 23:45:00  50.611111         p01
2019-11-01 23:50:00  50.131579         p01
2019-11-01 23:55:00  50.194444         p01
2019-11-02 00:00:00  51.636364         p01

[289 rows x 2 columns]


In [17]:
exercise_aggregated = aggregate_copy(
    dataframe=exercises, 
    group_minutes=group_minutes,
    day=day
)

print(exercise_aggregated)

                     activeDuration activityName participant
startTime                                                   
2019-11-01 14:55:00         1331000         Walk         p01
2019-11-01 19:05:00         2202000         Walk         p01


In [18]:
exercise_aggregated['activeDuration'] = pd.to_timedelta(exercise_aggregated['activeDuration'], unit='ms')

# Add duration to start_time to get end_time
exercise_aggregated['endTime'] = exercise_aggregated.index + exercise_aggregated['activeDuration']
exercise_aggregated = exercise_aggregated.drop(columns='activeDuration')

print(exercise_aggregated)

                    activityName participant             endTime
startTime                                                       
2019-11-01 14:55:00         Walk         p01 2019-11-01 15:17:11
2019-11-01 19:05:00         Walk         p01 2019-11-01 19:41:42


In [19]:
result = pd.merge_asof(calories_aggregated, bpm_aggregated, on='dateTime')
result = pd.merge(result, exercise_aggregated, left_on='dateTime', right_on='startTime', how="outer")
result = result[['dateTime','value_x', 'participant_x', 'value_y', 'activityName', 'endTime']]

print(result)

               dateTime  value_x participant_x    value_y activityName endTime
0   2019-11-01 00:00:00     4.17           p01  53.000000          NaN     NaT
1   2019-11-01 00:05:00     6.95           p01  53.488889          NaN     NaT
2   2019-11-01 00:10:00     6.95           p01  53.277778          NaN     NaT
3   2019-11-01 00:15:00     6.95           p01  54.171429          NaN     NaT
4   2019-11-01 00:20:00     6.95           p01  54.000000          NaN     NaT
..                  ...      ...           ...        ...          ...     ...
284 2019-11-01 23:40:00     6.95           p01  55.850000          NaN     NaT
285 2019-11-01 23:45:00    11.26           p01  50.611111          NaN     NaT
286 2019-11-01 23:50:00    24.87           p01  50.131579          NaN     NaT
287 2019-11-01 23:55:00    22.52           p01  50.194444          NaN     NaT
288 2019-11-02 00:00:00     2.92           p01  51.636364          NaN     NaT

[289 rows x 6 columns]


Loop that iterates each rows, copying the previous row, wheter there is an activity, if its timestamp (dateTime) is less than the endTime column.

In [20]:
for index, row in result.iterrows():
    if(index > 0):
        if (result.loc[index-1, 'endTime'] >= result.loc[index, 'dateTime']):
            result.loc[index, 'endTime']  = result.loc[index-1, 'endTime'] 
            result.loc[index, 'activityName'] = result.loc[index-1, 'activityName']


In [23]:
print(result)

               dateTime  value_x participant_x    value_y activityName  \
0   2019-11-01 00:00:00     4.17           p01  53.000000          NaN   
1   2019-11-01 00:05:00     6.95           p01  53.488889          NaN   
2   2019-11-01 00:10:00     6.95           p01  53.277778          NaN   
3   2019-11-01 00:15:00     6.95           p01  54.171429          NaN   
4   2019-11-01 00:20:00     6.95           p01  54.000000          NaN   
..                  ...      ...           ...        ...          ...   
284 2019-11-01 23:40:00     6.95           p01  55.850000          NaN   
285 2019-11-01 23:45:00    11.26           p01  50.611111          NaN   
286 2019-11-01 23:50:00    24.87           p01  50.131579          NaN   
287 2019-11-01 23:55:00    22.52           p01  50.194444          NaN   
288 2019-11-02 00:00:00     2.92           p01  51.636364          NaN   

    endTime  walking  swimming  running  cycling  treadmill  
0       NaT    False     False    False    False 

In [21]:
result['walking'] = result.apply(lambda row: True if (row['activityName'] == 'Walk') else False, axis=1)

result['swimming'] = result.apply(lambda row: True if (row['activityName'] == 'Swim') else False, axis=1)

result['running'] = result.apply(lambda row: True if (row['activityName'] == 'Run') else False, axis=1)

result['cycling'] = result.apply(lambda row: True if (row['activityName'] == 'Outdoor Bike') else False, axis=1)

result['treadmill'] = result.apply(lambda row: True if (row['activityName'] == 'Treadmill') else False, axis=1)


print(result)

               dateTime  value_x participant_x    value_y activityName  \
0   2019-11-01 00:00:00     4.17           p01  53.000000          NaN   
1   2019-11-01 00:05:00     6.95           p01  53.488889          NaN   
2   2019-11-01 00:10:00     6.95           p01  53.277778          NaN   
3   2019-11-01 00:15:00     6.95           p01  54.171429          NaN   
4   2019-11-01 00:20:00     6.95           p01  54.000000          NaN   
..                  ...      ...           ...        ...          ...   
284 2019-11-01 23:40:00     6.95           p01  55.850000          NaN   
285 2019-11-01 23:45:00    11.26           p01  50.611111          NaN   
286 2019-11-01 23:50:00    24.87           p01  50.131579          NaN   
287 2019-11-01 23:55:00    22.52           p01  50.194444          NaN   
288 2019-11-02 00:00:00     2.92           p01  51.636364          NaN   

    endTime  walking  swimming  running  cycling  treadmill  
0       NaT    False     False    False    False 

In [22]:
def insert_table(dataframe, group_minutes: int):
    command = """
            INSERT INTO public.group_{}_values (
                partecipant,
                VT,
                calories,
                bpm,
                walking,
                swimming,
                running,
                cycling,
                treadmill
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """.format(str(group_minutes))

    connection = None
    try:
        # connect to the PostgreSQL server
        connection = psycopg2.connect(
            database='postgres', 
            user='postgres', 
            password='pwd'
        )

        cur = connection.cursor()

        for index, row in dataframe.iterrows():

            participant = dataframe.loc[index, 'participant_x']
            participant = int(participant[1:])

            date = pd.to_datetime(dataframe.loc[index, 'dateTime'])
            calories = int(dataframe.loc[index, 'value_x'])
            bpm = int(dataframe.loc[index, 'value_y'])
            

            cur.execute(command, (
                participant, 
                date, 
                calories,
                bpm,
                bool(dataframe.loc[index, 'walking']),
                bool(dataframe.loc[index, 'swimming']),
                bool(dataframe.loc[index, 'cycling']),  
                bool(dataframe.loc[index, 'running']),
                bool(dataframe.loc[index, 'treadmill'])                
                )
            )
            
        cur.close()
        
        # commit the changes
        connection.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

    finally:
        if connection is not None:
            connection.close()
            
    return

insert_table(dataframe=result, group_minutes=group_minutes)

duplicate key value violates unique constraint "group_5_values_pkey"
DETAIL:  Key (partecipant, vt)=(1, 2019-11-01 00:00:00) already exists.

